In [2]:
import sqlite3
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval
from urllib.request import urlopen
from json import load
sns.set_style('white')
sns.set_context('notebook')

# 1. Imports

In [6]:
conn = sqlite3.connect('./Data/knwlg_blf.db')
subjs = pd.read_sql("SELECT * FROM subjects WHERE participation_date > 8/24/2022 "
                   "AND block1_complete=TRUE "
                   "AND block2_complete=TRUE "
                   "AND block3_complete=TRUE "
                   "AND completion_code LIKE '%XXX%'", con=conn)